In [1]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple
import pandas as pd
from collections import defaultdict
import hashlib, random, math
import os

# Constants 

In [2]:
SLOTS_PER_EPOCH = 2400
SLOT_DURATION_SEC = 30
EEP_FRACTION = 1/3
EEP_SLOT = int(SLOTS_PER_EPOCH * EEP_FRACTION)
PENALTY_FACTOR = 2.0
RECOVERY_FACTOR = 1.0
TOLERATED_BAD_RATIO = 0.02
REWARD_PER_BLOCK = 0.0001
AUTO_COMPOUND = True
RNG_SEED = 42
MIN_BET_AMOUNT = 50.0

# Classes

In [3]:
@dataclass
class Wallet:
    address: str
    green: float = 0.0
    red: float   = 0.0

    def get_balance(self) -> Dict[str, float]:
        """Ritorna i saldi correnti (green=stake, red=fee)."""
        return {"green": self.green, "red": self.red}

    def deposit(self, green: float = 0.0, red: float = 0.0) -> None:
        """Ricarica bilanci (stake/fee)."""
        if green > 0: self.green += green
        if red   > 0: self.red   += red

    def transfer(self, other: "Wallet", *, green_amount: float = 0.0, red_amount: float = 0.0) -> bool:
        """
        Trasferisce token verso un altro wallet (come nel wallet reale: indichi green e red).
        Metti 0 se vuoi trasferire solo uno dei due.
        """
        if green_amount < 0 or red_amount < 0:
            return False
        if green_amount > self.green or red_amount > self.red:
            return False

        if green_amount:
            self.green -= green_amount
            other.green += green_amount
        if red_amount:
            self.red -= red_amount
            other.red += red_amount
        return (green_amount > 0) or (red_amount > 0)

    def pay_fee(self, amount: float) -> bool:
        """
        Paga una fee consumando prima RED (TKR) e solo se insufficiente usa GREEN (TKG).
        """
        if amount <= 0:
            return True

        use_red = min(self.red, amount)
        self.red -= use_red
        remaining = amount - use_red
        if remaining <= 0:
            return True

        if remaining > self.green:
            self.green = max(0.0, self.green - remaining)
            return False
        self.green -= remaining
        return True

In [4]:
@dataclass
class Stakeholder(Wallet):
    display_name: Optional[str] = None

    def bet_op(self, to_main_id: str, amount: float, when_slot: int) -> dict:
        return {"kind": "STAKE", "from": self.address, "to": to_main_id,
                "amount": float(amount), "slot": when_slot}

In [5]:
@dataclass
class MainAddress(Wallet):
    delegates: Dict[str, float] = field(default_factory=dict)

    active_stake: float = 0.0

    overflows: Dict[str, "OverflowNode"] = field(default_factory=dict)

    def register_overflow(self, overflow: "OverflowNode") -> None:
        """
        Registra un overflow associato a questo main.
        Lo aggiunge al dizionario 'overflows' e aggiorna il main_id del nodo.
        """
        overflow.main_id = self.address
        self.overflows[overflow.address] = overflow

    def get_overflow_list(self) -> list:
        """Ritorna la lista degli overflow associati a questo main."""
        return list(self.overflows.values())

    def bet(self, from_address: str, amount: float) -> bool:
        """
        Riceve una delega (bet) in GREEN da uno stakeholder o dal main stesso.
        La quantità deve superare MIN_BET_AMOUNT.
        """
        if amount < MIN_BET_AMOUNT:
            return False
        self.delegates[from_address] = self.delegates.get(from_address, 0.0) + amount
        return True

    def self_bet(self, amount: float) -> bool:
        """
        Il main punta su sé stesso (auto-stake).
        Scala i suoi green e registra la delega come '_self_'.
        """
        if amount < MIN_BET_AMOUNT or amount > self.green:
            return False
        self.green -= amount
        return self.bet("_self_", amount)

    def self_unbet(self, amount: Optional[float] = None) -> float:
        """
        Revoca la self-bet del MAIN (tutta o parziale).
        Sposta dalla voce delegates['_self_'] ai green del main.
        :param amount: None => ritira TUTTA la self-bet; altrimenti ritira 'amount' (cappato al disponibile).
        :return: quantità effettivamente ritirata.
        """
        cur = self.delegates.get("_self_", 0.0)
        if cur <= 0:
            return 0.0
        take = cur if amount is None else max(0.0, min(amount, cur))
        if take <= 0:
            return 0.0

        residual = cur - take
        if residual > 0:
            self.delegates["_self_"] = residual
        else:
            self.delegates.pop("_self_", None)

        self.green += take
        return take

    def total_delegated_now(self) -> float:
        """Somma di tutte le deleghe correnti (inclusi self-bet e reward composte)."""
        return sum(self.delegates.values())

    def snapshot_at_eep(self) -> None:
        """Congela la stake attiva per l’epoch successivo."""
        self.active_stake = self.total_delegated_now()

    def credit_reward(self, amount: float) -> None:
        """
        Distribuisce la reward proporzionalmente alle deleghe correnti,
        componendola subito nella stake di ciascun delegante.
        """
        if amount <= 0:
            return
        total = self.total_delegated_now()
        if total <= 0:
            self.green += amount
            return

        for addr, stake_amt in list(self.delegates.items()):
            share = amount * (stake_amt / total)
            self.delegates[addr] = stake_amt + share

In [6]:
@dataclass
class OverflowNode(Wallet):
    main_id: str = ""
    base_stake_share: float = 1.0

    penalty_score: float = 0.0
    p_skip: float = 0.00
    p_invalid: float = 0.00
    p_delayed: float = 0.00

    effective_stake: float = 0.0
    assigned_slots: int = 0

    def behavior_outcome(self, rng) -> str:
        """Esito di uno slot (stocastico) in base ai parametri di affidabilità."""
        x = rng.random()
        if x < self.p_skip:    return "skipped"
        x -= self.p_skip
        if x < self.p_invalid: return "invalid"
        x -= self.p_invalid
        if x < self.p_delayed: return "delayed"
        return "ok"

    def slash_green(self, amount: float) -> float:
        """
        Applica una penalità sul COLLATERAL (GREEN) del nodo.
        Ritorna quanto è stato effettivamente tagliato.
        """
        if amount <= 0:
            return 0.0
        taken = min(self.green, amount)
        self.green -= taken
        return taken

In [7]:
from __future__ import annotations
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
import math, hashlib
import pandas as pd

class TakamakaSim:
    def __init__(self,
                 *,
                 slots_per_epoch: int = SLOTS_PER_EPOCH,
                 slot_duration_sec: int = SLOT_DURATION_SEC,
                 eep_fraction: float = EEP_FRACTION,
                 penalty_factor: float = PENALTY_FACTOR,
                 recovery_factor: float = RECOVERY_FACTOR,
                 tolerated_bad_ratio: float = TOLERATED_BAD_RATIO,
                 reward_per_block: float = REWARD_PER_BLOCK,
                 seed: int = RNG_SEED,
                 slash_per_invalid: float = 0.0):
        self.cfg = {
            "slots_per_epoch": int(slots_per_epoch),
            "slot_duration_sec": int(slot_duration_sec),
            "eep_slot": int(slots_per_epoch * eep_fraction),  # EEP_t in [0 .. S-1]
            "penalty_factor": float(penalty_factor),
            "recovery_factor": float(recovery_factor),
            "tolerated_bad_ratio": float(tolerated_bad_ratio),
            "reward_per_block": float(reward_per_block),
            "seed": int(seed),
            "slash_per_invalid": float(slash_per_invalid),
        }
        self.epoch = 0

        # registri
        self.stakeholders: Dict[str, Stakeholder] = {}
        self.mains: Dict[str, MainAddress] = {}
        self.overflows: Dict[str, OverflowNode] = {}

        # op differite → maturano nella finestra (EEP_{t-1}, EEP_t]
        self.pending_ops: List[Dict] = []

        # storico KPI
        self.history: List[Dict] = []

        # pesi “freezati” per la distribuzione slot
        self.weights_for_epoch: Dict[str, float] = {}   # usati per distribuire l'epoca corrente t
        self.weights_next_epoch: Dict[str, float] = {}  # calcolati all'EEP di t, usati in t+1
        self._initialized_weights = False

        # bootstrap speciale per epoca 0
        self._bootstrap_done = False

    # ---------------------------
    # Registrazione / indexing
    # ---------------------------
    def register_stakeholder(self, st: Stakeholder):
        self.stakeholders[st.address] = st

    def register_main(self, m: MainAddress):
        self.mains[m.address] = m

    def index_overflow(self, node: OverflowNode):
        if node.main_id not in self.mains:
            raise ValueError("main_id non registrato")
        self.overflows[node.address] = node

    # ---------------------------
    # Operazioni differite (EEP)
    # ---------------------------
    def queue_bet(self, from_addr: str, to_main_id: str, amount: float, when_slot: int):
        if to_main_id not in self.mains:
            raise ValueError(f"Main non registrato: {to_main_id}")
        if from_addr not in self.stakeholders and from_addr != "_self_":
            raise ValueError(f"Stakeholder non registrato: {from_addr}")
        if amount < MIN_BET_AMOUNT:
            raise ValueError(f"Importo minimo di puntata è {MIN_BET_AMOUNT}, got {amount}")
        self.pending_ops.append({
            "kind": "STAKE",
            "from": from_addr,
            "to": to_main_id,
            "amount": float(amount),
            "epoch": self.epoch,
            "slot": int(when_slot),
        })

    def queue_stake_undo(self, from_addr: str, when_slot: int):
        if from_addr not in self.stakeholders:
            raise ValueError(f"Stakeholder non registrato: {from_addr}")
        self.pending_ops.append({
            "kind": "STAKE_UNDO",
            "from": from_addr,
            "epoch": self.epoch,
            "slot": int(when_slot),
        })

    def queue_self_unbet(self, main_id: str, when_slot: int, amount: Optional[float] = None):
        if main_id not in self.mains:
            raise ValueError(f"Main non registrato: {main_id}")
        self.pending_ops.append({
            "kind": "SELF_UNBET",
            "to": main_id,
            "amount": None if amount is None else float(amount),
            "epoch": self.epoch,
            "slot": int(when_slot),
        })

    # ---------------------------
    # Finestra EEP: applicazione op che maturano ORA (per preparare t+1)
    # ---------------------------
    def _apply_ops_at_current_eep(self):
        eep = self.cfg["eep_slot"]
        eligible, keep = [], []
        for op in self.pending_ops:
            if (op["epoch"] == self.epoch and op["slot"] <= eep) or \
               (op["epoch"] == self.epoch - 1 and op["slot"] > eep):
                eligible.append(op)
            else:
                keep.append(op)

        # Ordine: prima revoche (UNDO/SELF_UNBET), poi STAKE
        def op_key(op):
            return (
                0 if op["kind"] in ("STAKE_UNDO", "SELF_UNBET") else 1,
                op.get("epoch", -1),
                op.get("slot", -1),
                op.get("from", ""),
                op.get("to", "")
            )
        eligible.sort(key=op_key)

        for op in eligible:
            kind = op["kind"]
            if kind == "STAKE_UNDO":
                deleg_addr = op["from"]
                st = self.stakeholders.get(deleg_addr)
                if st is None:
                    continue
                for m in self.mains.values():
                    cur = m.delegates.get(deleg_addr, 0.0)
                    if cur > 0:
                        st.green += cur
                        m.delegates.pop(deleg_addr, None)

            elif kind == "STAKE":
                to_main = self.mains.get(op["to"])
                if to_main is None:
                    continue
                amount = float(op["amount"])
                if amount < MIN_BET_AMOUNT:
                    continue
                from_addr = op["from"]

                if from_addr == "_self_":
                    if amount <= to_main.green:
                        to_main.green -= amount
                        to_main.bet("_self_", amount)
                else:
                    st = self.stakeholders.get(from_addr)
                    if st is None:
                        continue
                    if amount <= st.green:
                        st.green -= amount
                        to_main.bet(from_addr, amount)

            elif kind == "SELF_UNBET":
                to_main = self.mains.get(op["to"])
                if to_main is None:
                    continue
                amt = op.get("amount", None)
                to_main.self_unbet(amt)

        self.pending_ops = keep

    # ---------------------------
    # Bootstrap speciale per epoca 0
    # ---------------------------
    def _bootstrap_epoch0_if_needed(self):
        """
        Se all'inizio dell'epoca 0 i pesi sono tutti zero, applichiamo SOLO
        le op con (epoch==0, slot==0) PRIMA della prima distribuzione, così
        qualcuno ha stake e può validare blocchi nell'epoca 0.
        Queste op vengono rimosse da pending_ops (non verranno riapplicate).
        """
        if self.epoch != 0 or self._bootstrap_done:
            return

        # calcola pesi iniziali (stato “genesis”)
        self.weights_for_epoch = self._compute_effective_weights_now()
        if sum(self.weights_for_epoch.values()) > 0:
            # c'è già stake: niente bootstrap
            self._initialized_weights = True
            self._bootstrap_done = True
            return

        # altrimenti, applica SOLO (epoch==0, slot==0)
        eligible, keep = [], []
        for op in self.pending_ops:
            if op["epoch"] == 0 and op["slot"] == 0:
                eligible.append(op)
            else:
                keep.append(op)

        # stesso ordinamento delle EEP
        def op_key(op):
            return (
                0 if op["kind"] in ("STAKE_UNDO", "SELF_UNBET") else 1,
                op.get("epoch", -1),
                op.get("slot", -1),
                op.get("from", ""),
                op.get("to", "")
            )
        eligible.sort(key=op_key)

        for op in eligible:
            kind = op["kind"]
            if kind == "STAKE_UNDO":
                deleg_addr = op["from"]
                st = self.stakeholders.get(deleg_addr)
                if st is None:
                    continue
                for m in self.mains.values():
                    cur = m.delegates.get(deleg_addr, 0.0)
                    if cur > 0:
                        st.green += cur
                        m.delegates.pop(deleg_addr, None)
            elif kind == "STAKE":
                to_main = self.mains.get(op["to"])
                if to_main is None:
                    continue
                amount = float(op["amount"])
                if amount < MIN_BET_AMOUNT:
                    continue
                from_addr = op["from"]
                if from_addr == "_self_":
                    if amount <= to_main.green:
                        to_main.green -= amount
                        to_main.bet("_self_", amount)
                else:
                    st = self.stakeholders.get(from_addr)
                    if st is None:
                        continue
                    if amount <= st.green:
                        st.green -= amount
                        to_main.bet(from_addr, amount)
            elif kind == "SELF_UNBET":
                to_main = self.mains.get(op["to"])
                if to_main is None:
                    continue
                amt = op.get("amount", None)
                to_main.self_unbet(amt)

        # aggiorna pending_ops (rimuove quelle consumate nel bootstrap)
        self.pending_ops = keep

        # >>> FIX: snapshot PRIMA di calcolare i pesi, così active_stake è valorizzata
        self._snapshot_all_mains_at_eep()
        self.weights_for_epoch = self._compute_effective_weights_now()

        self._initialized_weights = True
        self._bootstrap_done = True

    # ---------------------------
    # Snapshot EEP (fotografa i main)
    # ---------------------------
    def _snapshot_all_mains_at_eep(self):
        for m in self.mains.values():
            m.snapshot_at_eep()

    # ---------------------------
    # Helpers deterministici
    # ---------------------------
    @staticmethod
    def _hash_u64(*parts: str) -> int:
        h = hashlib.sha256("::".join(parts).encode()).digest()
        return int.from_bytes(h[:8], "big", signed=False)

    def _epoch_seed_str(self) -> str:
        return f"takamaka-seed:{self.cfg['seed']}:{self.epoch}"

    @staticmethod
    def _effective_from_penalty(base: float, penalty_score: float) -> float:
        return base / (1.0 + max(0.0, penalty_score))

    def _quota_counts(self, weights: Dict[str, float]) -> Dict[str, int]:
        S = self.cfg["slots_per_epoch"]
        total = sum(weights.values())
        if total <= 0:
            return {k: 0 for k in weights}
        expected = {k: (w / total) * S for k, w in weights.items()}
        base = {k: int(math.floor(v)) for k, v in expected.items()}
        remainder = S - sum(base.values())

        seed = self._epoch_seed_str()
        frac = []
        for k, v in expected.items():
            frac_part = v - math.floor(v)
            tiebreak = self._hash_u64(seed, "frac", k)
            frac.append((k, frac_part, tiebreak))
        frac.sort(key=lambda x: (-x[1], x[2]))
        for i in range(remainder):
            base[frac[i][0]] += 1
        return base

    # ---------------------------
    # Calcolo pesi effettivi dallo stato attuale
    # ---------------------------
    def _compute_effective_weights_now(self) -> Dict[str, float]:
        weights: Dict[str, float] = {}
        for node in self.overflows.values():
            main = self.mains[node.main_id]
            base = max(0.0, main.active_stake * max(0.0, node.base_stake_share))
            eff  = self._effective_from_penalty(base, node.penalty_score)
            weights[node.address] = eff
        return weights

    def _ensure_weights_initialized(self):
        if self._initialized_weights:
            return
        # Se il bootstrap ha già impostato i pesi per t=0, non ricalcolare
        if self.epoch == 0 and self._bootstrap_done:
            self._initialized_weights = True
            return
        # Stato corrente
        self.weights_for_epoch = self._compute_effective_weights_now()
        # Se zero e siamo in epoca 0, prova il bootstrap
        if self.epoch == 0 and sum(self.weights_for_epoch.values()) == 0:
            self._bootstrap_epoch0_if_needed()
            return
        self._initialized_weights = True

    # ---------------------------
    # Assegnazione slot con pesi forniti (freezati)
    # ---------------------------
    def _distribute_slots_with_weights(self, weights: Dict[str, float]) -> Tuple[Dict[str, int], List[str]]:
        for addr, node in self.overflows.items():
            node.effective_stake = float(weights.get(addr, 0.0))

        counts = self._quota_counts(weights)

        for addr, cnt in counts.items():
            self.overflows[addr].assigned_slots = cnt

        seed = self._epoch_seed_str()
        tickets = []
        for addr, cnt in counts.items():
            for k in range(cnt):
                t = self._hash_u64(seed, "slot", addr, str(k))
                tickets.append((t, addr))
        tickets.sort(key=lambda x: x[0])
        schedule = [addr for _, addr in tickets]
        return counts, schedule

    # ---------------------------
    # Penalità e reward
    # ---------------------------
    def _apply_penalties_and_rewards(self, per_node_outcomes: Dict[str, Dict[str, int]]):
        for addr, outcome in per_node_outcomes.items():
            node = self.overflows[addr]
            main = self.mains[node.main_id]

            assigned = max(1, node.assigned_slots)  # evita /0
            bad = outcome.get("skipped", 0) + outcome.get("invalid", 0) + outcome.get("delayed", 0)
            ok_blocks = outcome.get("ok", 0)
            bad_ratio = bad / assigned

            # penalità/recovery
            if bad_ratio > self.cfg["tolerated_bad_ratio"]:
                node.penalty_score += self.cfg["penalty_factor"] * (bad_ratio - self.cfg["tolerated_bad_ratio"])
            else:
                node.penalty_score = max(
                    0.0,
                    node.penalty_score - self.cfg["recovery_factor"] * (self.cfg["tolerated_bad_ratio"] - bad_ratio)
                )

            # slashing su invalid
            spiv = self.cfg["slash_per_invalid"]
            if spiv > 0.0:
                slash = outcome.get("invalid", 0) * spiv
                if slash > 0:
                    node.slash_green(slash)

            # reward costante, compounding immediato
            reward = self.cfg["reward_per_block"] * ok_blocks
            main.credit_reward(reward)

    # ---------------------------
    # Prepara i pesi per (t+1) all'EEP di t
    # ---------------------------
    def _compute_weights_for_next_epoch_at_current_eep(self):
        self._apply_ops_at_current_eep()
        self._snapshot_all_mains_at_eep()
        self.weights_next_epoch = self._compute_effective_weights_now()

    # ---------------------------
    # Simulazione epoch
    # ---------------------------
    def simulate_one_epoch(self) -> Dict:
        """
        Sequenza:
          A) (solo per t=0) bootstrap se necessario per evitare 0 slot,
          B) usa i pesi freezati (weights_for_epoch) per distribuire l'epoca t,
          C) simula gli slot,
          D) applica penalità/reward,
          E) all'EEP di t prepara i pesi per t+1,
          F) roll: weights_for_epoch = weights_next_epoch, t ← t+1.
        """
        # A) bootstrap/inizializzazione pesi
        if self.epoch == 0 and not self._bootstrap_done:
            self._bootstrap_epoch0_if_needed()
        self._ensure_weights_initialized()

        # B) distribuzione epoca t con pesi freezati
        slots_per_node, schedule = self._distribute_slots_with_weights(self.weights_for_epoch)

        # C) simulazione slot
        from collections import defaultdict as _dd
        import random as _rnd
        per_node_outcomes = _dd(lambda: {"ok": 0, "skipped": 0, "invalid": 0, "delayed": 0})
        rng_slots = _rnd.Random(self.cfg["seed"] * 7919 + self.epoch)
        for addr in schedule:
            outcome = self.overflows[addr].behavior_outcome(rng_slots)
            per_node_outcomes[addr][outcome] += 1

        # D) penalità + reward
        self._apply_penalties_and_rewards(per_node_outcomes)

        # E) EEP di t → pesi per t+1
        self._compute_weights_for_next_epoch_at_current_eep()

        # KPI (coerenti con i pesi usati in t)
        total_effective_stake = sum(self.weights_for_epoch.values())
        total_slots_epoch = int(self.cfg["slots_per_epoch"])
        kpi = {
            "epoch": self.epoch,
            "slots_per_node": dict(slots_per_node),
            "per_node_outcomes": {addr: dict(v) for addr, v in per_node_outcomes.items()},
            "penalties": {addr: self.overflows[addr].penalty_score for addr in self.overflows},
            "effective_stake_per_overflow": {addr: self.overflows[addr].effective_stake for addr in self.overflows},
            "total_effective_stake": total_effective_stake,
            "total_slots": total_slots_epoch,
        }
        self.history.append(kpi)

        # F) roll epoca
        self.weights_for_epoch = dict(self.weights_next_epoch)
        self.epoch += 1
        return kpi

    # ---------------------------
    # Report
    # ---------------------------
    def epoch_summary_df(self, kpi: Dict) -> pd.DataFrame:
        total_effective = kpi.get("total_effective_stake", 0.0)
        total_slots_epoch = int(kpi.get("total_slots", 0))
        rows = []

        for addr, slots in kpi["slots_per_node"].items():
            o = kpi["per_node_outcomes"].get(addr, {})
            node = self.overflows[addr]

            ok = int(o.get("ok", 0))
            skipped = int(o.get("skipped", 0))
            invalid = int(o.get("invalid", 0))
            delayed = int(o.get("delayed", 0))

            bad_abs = skipped + invalid + delayed

            stake_share_pct = (node.effective_stake / total_effective * 100.0) if total_effective > 0 else 0.0
            delivered_slots_share_pct = (ok / total_slots_epoch * 100.0) if total_slots_epoch > 0 else 0.0
            bad_slots_pct_node = (bad_abs / slots * 100.0) if slots > 0 else 0.0

            rows.append({
                "epoch": kpi["epoch"],
                "overflow": addr,
                "main": node.main_id,
                "assigned_slots": int(slots),

                "ok": ok,
                "skipped": skipped,
                "invalid": invalid,
                "delayed": delayed,

                "penalty_score": node.penalty_score,
                "effective_stake": node.effective_stake,

                "stake_share_pct": round(stake_share_pct, 2),
                "delivered_slots_share_pct": round(delivered_slots_share_pct, 2),
                "bad_slots_abs": int(bad_abs),
                "bad_slots_pct_node": round(bad_slots_pct_node, 2),
            })

        return pd.DataFrame(rows)


# Simulations

## Simulation 1

This simulation runs for 20 epochs.
- During epochs 0–4, all stake is placed on NODE1, so NODE1 receives every slot.
- From epochs 5–19, the stake is redistributed across the three nodes in proportions of 20% (NODE1), 30% (NODE2), and 50% (NODE3).
- All nodes behave correctly, meaning that every assigned slot is validated as “ok.”

The final report is saved to ./data/simulation_1.csv.

In [8]:
# --- setup simulazione ---
sim = TakamakaSim(slots_per_epoch=SLOTS_PER_EPOCH, seed=RNG_SEED)

# registra 3 main
sim.register_main(MainAddress(address="NODE1", active_stake=0.0))
sim.register_main(MainAddress(address="NODE2", active_stake=0.0))
sim.register_main(MainAddress(address="NODE3", active_stake=0.0))

# indicizza 3 overflow (1:1)
sim.index_overflow(OverflowNode(address="NODE1", main_id="NODE1", base_stake_share=1.0))
sim.index_overflow(OverflowNode(address="NODE2", main_id="NODE2", base_stake_share=1.0))
sim.index_overflow(OverflowNode(address="NODE3", main_id="NODE3", base_stake_share=1.0))

# --- Stakeholder iniziale: 100 GREEN su NODE1 ---
st1 = Stakeholder(address="ST1"); st1.green = 100.0
sim.register_stakeholder(st1)

# BET che matura all'EEP dell'epoch corrente (effetto immediato alla prossima simulate_one_epoch)
sim.queue_bet(from_addr="ST1", to_main_id="NODE1", amount=100.0, when_slot=0)

# --- Simula 4 epoche (stake solo su NODE1) ---
all_rows: List[pd.DataFrame] = []
for _ in range(4):
    kpi = sim.simulate_one_epoch()
    all_rows.append(sim.epoch_summary_df(kpi))

# --- Nuovi stakeholder: 150 su NODE2, 250 su NODE3 ---
st2 = Stakeholder(address="ST2"); st2.green = 150.0
st3 = Stakeholder(address="ST3"); st3.green = 250.0
sim.register_stakeholder(st2)
sim.register_stakeholder(st3)

# BET che maturano all'EEP dell'epoch corrente (da ora in poi NODE2/NODE3 entrano con la loro stake)
sim.queue_bet(from_addr="ST2", to_main_id="NODE2", amount=150.0, when_slot=0)
sim.queue_bet(from_addr="ST3", to_main_id="NODE3", amount=250.0, when_slot=0)

# --- Simula altre 16 epoche per arrivare a 20 in totale ---
for _ in range(16):
    kpi = sim.simulate_one_epoch()
    all_rows.append(sim.epoch_summary_df(kpi))

# --- Salva report ---
report_df = pd.concat(all_rows, ignore_index=True)
out_path = "./data/simulation_1.csv"
report_df.to_csv(out_path, index=False)

## Simulation 2

This simulation also runs for 20 epochs.  
- During epochs 0–4, all stake is placed on NODE1, so NODE1 receives every slot.  
- From epochs 5–19, the stake is redistributed across the three nodes in proportions of 20% (NODE1), 30% (NODE2), and 50% (NODE3).  
- NODE1 behaves maliciously, with a 10% probability of skipping its assigned slots, while NODE2 and NODE3 behave correctly and always validate their slots as “ok.”  

The final report is saved to ./data/simulation_2.csv.

In [9]:
# setup simulazione 2
sim2 = TakamakaSim(slots_per_epoch=SLOTS_PER_EPOCH, seed=RNG_SEED)

# registra 3 main
sim2.register_main(MainAddress(address="NODE1", active_stake=0.0))
sim2.register_main(MainAddress(address="NODE2", active_stake=0.0))
sim2.register_main(MainAddress(address="NODE3", active_stake=0.0))

# indicizza 3 overflow (1:1)
# NODE1 malevolo: 10% skipped; invalid/delayed a 0
sim2.index_overflow(OverflowNode(address="NODE1", main_id="NODE1", base_stake_share=1.0,
                                 p_skip=0.10, p_invalid=0.00, p_delayed=0.00))
# NODE2 e NODE3 regolari (0 su tutte le probabilità => sempre "ok")
sim2.index_overflow(OverflowNode(address="NODE2", main_id="NODE2", base_stake_share=1.0,
                                 p_skip=0.00, p_invalid=0.00, p_delayed=0.00))
sim2.index_overflow(OverflowNode(address="NODE3", main_id="NODE3", base_stake_share=1.0,
                                 p_skip=0.00, p_invalid=0.00, p_delayed=0.00))

# --- Stakeholder iniziale: 100 GREEN su NODE1 ---
st1 = Stakeholder(address="ST2_STAKE1"); st1.green = 100.0  # id diverso per separare dalla sim1
sim2.register_stakeholder(st1)

# BET che matura all'EEP dell'epoch corrente (effetto alla prossima simulate_one_epoch)
sim2.queue_bet(from_addr="ST2_STAKE1", to_main_id="NODE1", amount=100.0, when_slot=0)

# --- Simula 4 epoche (stake solo su NODE1) ---
all_rows_2: List[pd.DataFrame] = []
for _ in range(4):
    kpi = sim2.simulate_one_epoch()
    all_rows_2.append(sim2.epoch_summary_df(kpi))

# --- Nuovi stakeholder: 150 su NODE2, 250 su NODE3 ---
st2 = Stakeholder(address="ST2_STAKE2"); st2.green = 150.0
st3 = Stakeholder(address="ST2_STAKE3"); st3.green = 250.0
sim2.register_stakeholder(st2)
sim2.register_stakeholder(st3)

# BET che maturano all'EEP dell'epoch corrente
sim2.queue_bet(from_addr="ST2_STAKE2", to_main_id="NODE2", amount=150.0, when_slot=0)
sim2.queue_bet(from_addr="ST2_STAKE3", to_main_id="NODE3", amount=250.0, when_slot=0)

# --- Simula altre 16 epoche per arrivare a 20 in totale ---
for _ in range(16):
    kpi = sim2.simulate_one_epoch()
    all_rows_2.append(sim2.epoch_summary_df(kpi))

# --- Salva report dedicato alla Simulazione 2 ---
report_df_2 = pd.concat(all_rows_2, ignore_index=True)
out_path_2 = "./data/simulation_2.csv"
report_df_2.to_csv(out_path_2, index=False)
